In [40]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [41]:
# Load the dataset using pandas
df = pd.read_csv('Titanic-Dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


**Note:** I will apply only the preprocessing steps that I performed manually in previous notebook, to see how much easier preprocessing becomes using a **ColumnTransformer**.

In [43]:
X = df.drop(columns=['Survived'])
y = df['Survived']

# Train split test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
331,332,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S
733,734,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S
382,383,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S
704,705,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S
813,814,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S


In [44]:
preprocessing = ColumnTransformer(
    transformers = [
    ('drop_cabin', 'drop', ['Cabin']),
    ('encode_sex', OneHotEncoder(drop='first', handle_unknown='ignore'), ['Sex']),
    ('impute_age', SimpleImputer(strategy='mean'), ['Age']),
    ('impute_encode_embarked', Pipeline(
        steps = [
            ('impute_embarked', SimpleImputer(strategy='most_frequent')),
            ('encode', OneHotEncoder(drop='first', handle_unknown='ignore'))
        ]
    ), ['Embarked'])
], remainder='passthrough')

In [45]:
X_train_preprocessed = preprocessing.fit_transform(X_train)
X_test_preprocessed = preprocessing.transform(X_test)

In [46]:
X_train_preprocessed

array([[1.0, 45.5, 0.0, 1.0, 332, 1, 'Partner, Mr. Austen', 0, 0, '113043', 28.5],
       [1.0, 23.0, 0.0, 1.0, 734, 2, 'Berriman, Mr. William John', 0, 0, '28425', 13.0],
       [1.0, 32.0, 0.0, 1.0, 383, 3, 'Tikkanen, Mr. Juho', 0, 0, 'STON/O 2. 3101293', 7.925],
       [1.0, 26.0, 0.0, 1.0, 705, 3, 'Hansen, Mr. Henrik Juul', 1, 0, '350025', 7.8542],
       [0.0, 6.0, 0.0, 1.0, 814, 3, 'Andersson, Miss. Ebba Iris Alfrida', 4, 2, '347082', 31.275],
       [1.0, 24.0, 0.0, 0.0, 119, 1, 'Baxter, Mr. Quigg Edmond', 0, 1, 'PC 17558', 247.5208],
       [1.0, 45.0, 0.0, 1.0, 537, 1, 'Butt, Major. Archibald Willingham', 0, 0, '113050', 26.55],
       [1.0, 29.0, 0.0, 0.0, 362, 2, 'del Carlo, Mr. Sebastiano', 1, 0, 'SC/PARIS 2167', 27.7208],
       [1.0, 29.498846153846156, 0.0, 1.0, 30, 3, 'Todoroff, Mr. Lalio', 0, 0, '349216', 7.8958],
       [1.0, 29.498846153846156, 0.0, 1.0, 56, 1, 'Woolner, Mr. Hugh', 0, 0, '19947', 35.5],
       [0.0, 42.0, 0.0, 1.0, 866, 2, 'Bystrom, Mrs. (Karolina)',

All preprocessing steps are combined into a single object, making the code clean and organized. We apply **fit_transform** only on the training data to prevent data leakage, ensuring the test data is transformed using the same fitted parameters.

This ensures clarity, code cleanliness, and the data leakage prevention.